In [ ]:
# use Discover API to download recent movie data

In [71]:
import os
import tmdbsimple as tmdb
import pandas as pd
import json
from tqdm import tqdm
import concurrent.futures

In [72]:
with open("../tmdb_key.txt") as f:
    tmdb.API_KEY = f.read().strip()

In [73]:
def movie_ids_by_year(target_year):

    def discover_ids_by_year(target_year=2015):
        discover = tmdb.Discover()
        all_results = []
        first_page = discover.movie(primary_release_year=target_year)
        print("Target year: {}".format(target_year))
        print("Total Results: {}".format(first_page['total_results']))
        for i in tqdm(range(1, first_page['total_pages']+1)):
            if i == 1:
                page = first_page
            else:
                page = discover.movie(primary_release_year=target_year, page=i)
            ids = [element['id'] for element in page['results']]
            all_results.extend(ids)

        print("Collected Results: {}".format(len(all_results)))
        print("Unique Results: {}".format(len(set(all_results))))
        with open("dataset/{}_ids.txt".format(target_year), "w") as f:
            f.write(",".join([str(k) for k in list(set(all_results))]))

    if not os.path.exists("dataset/{}_ids.txt".format(target_year)):
        discover_ids_by_year(target_year)

    with open("dataset/{}_ids.txt".format(target_year, "r")) as f:
        all_results = f.read().split(",")

    all_results = list(set(all_results))  # remove duplicates
    return all_results

In [74]:
def get_movie_info(ids, target_year):
    
    try:
        with open("dataset/{}_movies_dict.json".format(target_year), "r") as f:
            movies_dict = json.load(f)
    except:
        movies_dict = {}

    try:
        with open("dataset/{}_US_movies_dict.json".format(target_year), "r") as f:
            US_movies_dict = json.load(f)
    except:
        US_movies_dict = {}

    print(f"Movies: {len(movies_dict)}, US Movies: {len(US_movies_dict)}")
    
    def download_id(movie_id):
        movie_id = int(movie_id)
        if str(movie_id) in movies_dict:
            return None
        try:
            movie = tmdb.Movies(movie_id)
            movie_info = movie.info()
            usa = ("United States of America" in str(movie_info['production_countries']) ) 
            return (movie_id, movie_info, usa)
        except:
            print("Error on ID {}.".format(movie_id))
        return None
    
    print("Download movie info")
    with concurrent.futures.ThreadPoolExecutor(2) as executor:
        res = list(tqdm(executor.map(download_id, ids), total=len(ids)))

    res = [x for x in res if x is not None]
    
    for movie_id, info, usa in tqdm(res):
        movies_dict[str(movie_id)] = info
        if usa:
            US_movies_dict[str(movie_id)] = info
        
    def download_keyword(item):
        movie_id, movie_info = item
        if "keywords" in movie_info:
            return None
        try:
            movie = tmdb.Movies(int(movie_id))
            movie_info["keywords"] = movie.keywords()['keywords']
            return (movie_id, movie_info)
        except:
            print("Error on ID {}.".format(movie_id))
        return None
    
    with open("dataset/{}_movies_dict.json".format(target_year), "w") as f:
        json.dump(movies_dict, f)

    with open("dataset/{}_US_movies_dict.json".format(target_year), "w") as f:
        json.dump(US_movies_dict, f)

    print("Download keywords for US movies")
    with concurrent.futures.ThreadPoolExecutor(2) as executor:
        res = list(tqdm(executor.map(download_keyword, US_movies_dict.items()), total=len(US_movies_dict)))
        
    res = [x for x in res if x is not None]
    
    for movie_id, info in tqdm(res):
        US_movies_dict[str(movie_id)] = info
    
    with open("dataset/{}_US_movies_dict.json".format(target_year), "w") as f:
        json.dump(US_movies_dict, f)
        
    print(f"Movies: {len(movies_dict)}, US Movies: {len(US_movies_dict)}")

    return movies_dict, US_movies_dict

In [80]:
target_year = 2016

In [81]:
ids = movie_ids_by_year(target_year)



  0%|          | 0/730 [00:00<?, ?it/s]

Target year: 2016
Total Results: 14599




  0%|          | 2/730 [00:00<02:55,  4.14it/s]

  0%|          | 3/730 [00:00<03:23,  3.58it/s]

  1%|          | 4/730 [00:01<04:02,  2.99it/s]

  1%|          | 5/730 [00:01<04:43,  2.56it/s]

  1%|          | 6/730 [00:02<05:11,  2.32it/s]

  1%|          | 7/730 [00:02<05:30,  2.19it/s]

  1%|          | 8/730 [00:03<05:11,  2.32it/s]

  1%|          | 9/730 [00:03<04:56,  2.43it/s]

  1%|▏         | 10/730 [00:03<04:49,  2.48it/s]

  2%|▏         | 11/730 [00:04<04:59,  2.40it/s]

  2%|▏         | 12/730 [00:04<05:21,  2.24it/s]

  2%|▏         | 13/730 [00:05<06:22,  1.87it/s]

  2%|▏         | 14/730 [00:06<05:46,  2.07it/s]

  2%|▏         | 15/730 [00:06<05:26,  2.19it/s]

  2%|▏         | 16/730 [00:06<05:32,  2.15it/s]

  2%|▏         | 17/730 [00:07<05:11,  2.29it/s]

  2%|▏         | 18/730 [00:07<04:58,  2.38it/s]

  3%|▎         | 19/730 [00:08<04:55,  2.41it/s]

  3%|▎         | 20/730 [00:08<04:55,  2.40it/s]

  3%|▎         | 21/730 [00:08<04:50,  2.44it/s]

  3%|▎

 40%|███▉      | 290/730 [02:02<02:50,  2.59it/s]

 40%|███▉      | 291/730 [02:02<02:47,  2.63it/s]

 40%|████      | 292/730 [02:02<02:47,  2.62it/s]

 40%|████      | 293/730 [02:03<02:45,  2.63it/s]

 40%|████      | 294/730 [02:03<03:17,  2.20it/s]

 40%|████      | 295/730 [02:04<03:08,  2.31it/s]

 41%|████      | 296/730 [02:04<03:03,  2.36it/s]

 41%|████      | 297/730 [02:05<03:05,  2.34it/s]

 41%|████      | 298/730 [02:05<02:58,  2.42it/s]

 41%|████      | 299/730 [02:05<02:56,  2.45it/s]

 41%|████      | 300/730 [02:06<02:56,  2.43it/s]

 41%|████      | 301/730 [02:06<02:53,  2.48it/s]

 41%|████▏     | 302/730 [02:07<02:51,  2.50it/s]

 42%|████▏     | 303/730 [02:07<02:57,  2.40it/s]

 42%|████▏     | 304/730 [02:07<02:53,  2.46it/s]

 42%|████▏     | 305/730 [02:08<03:20,  2.12it/s]

 42%|████▏     | 306/730 [02:09<03:17,  2.14it/s]

 42%|████▏     | 307/730 [02:09<03:07,  2.26it/s]

 42%|████▏     | 308/730 [02:09<03:20,  2.10it/s]

 42%|████▏     | 309/730 [02:10

 79%|███████▉  | 576/730 [04:05<01:02,  2.46it/s]

 79%|███████▉  | 577/730 [04:05<01:00,  2.54it/s]

 79%|███████▉  | 578/730 [04:06<01:00,  2.53it/s]

 79%|███████▉  | 579/730 [04:06<01:01,  2.47it/s]

 79%|███████▉  | 580/730 [04:06<01:04,  2.34it/s]

 80%|███████▉  | 581/730 [04:07<01:01,  2.41it/s]

 80%|███████▉  | 582/730 [04:07<01:02,  2.38it/s]

 80%|███████▉  | 583/730 [04:08<01:00,  2.42it/s]

 80%|████████  | 584/730 [04:08<01:02,  2.35it/s]

 80%|████████  | 585/730 [04:09<01:00,  2.38it/s]

 80%|████████  | 586/730 [04:09<00:59,  2.42it/s]

 80%|████████  | 587/730 [04:09<00:58,  2.44it/s]

 81%|████████  | 588/730 [04:10<00:58,  2.42it/s]

 81%|████████  | 589/730 [04:10<00:56,  2.48it/s]

 81%|████████  | 590/730 [04:11<00:55,  2.51it/s]

 81%|████████  | 591/730 [04:11<00:55,  2.52it/s]

 81%|████████  | 592/730 [04:11<00:54,  2.51it/s]

 81%|████████  | 593/730 [04:12<00:55,  2.45it/s]

 81%|████████▏ | 594/730 [04:12<01:01,  2.20it/s]

 82%|████████▏ | 595/730 [04:13

Collected Results: 14599
Unique Results: 11162


In [ ]:
movies_dict, US_movies_dict = get_movie_info(ids, target_year)

Movies: 0, US Movies: 0
Download movie info




  0%|          | 0/11162 [00:00<?, ?it/s]

  0%|          | 1/11162 [00:00<1:04:43,  2.87it/s]

  0%|          | 3/11162 [00:00<56:59,  3.26it/s]  

  0%|          | 5/11162 [00:01<50:44,  3.66it/s]

  0%|          | 7/11162 [00:01<46:42,  3.98it/s]

  0%|          | 9/11162 [00:01<43:50,  4.24it/s]

  0%|          | 10/11162 [00:02<39:21,  4.72it/s]

  0%|          | 11/11162 [00:02<41:53,  4.44it/s]

  0%|          | 12/11162 [00:02<42:29,  4.37it/s]

  0%|          | 13/11162 [00:02<38:38,  4.81it/s]

  0%|          | 14/11162 [00:03<42:12,  4.40it/s]

  0%|          | 15/11162 [00:03<37:48,  4.91it/s]

  0%|          | 16/11162 [00:03<44:50,  4.14it/s]

  0%|          | 17/11162 [00:03<37:11,  4.99it/s]

  0%|          | 18/11162 [00:03<42:28,  4.37it/s]

  0%|          | 19/11162 [00:04<38:44,  4.79it/s]

  0%|          | 20/11162 [00:04<39:36,  4.69it/s]

  0%|          | 21/11162 [00:04<39:16,  4.73it/s]

  0%|          | 22/11162 [00:04<35:48,  5.18it/s]

  0%|          | 23/

Error on ID 544516.




  0%|          | 44/11162 [00:09<42:33,  4.35it/s]

Error on ID 409239.
Error on ID 398237.


...



 63%|██████▎   | 7042/11162 [28:10<20:57,  3.28it/s]

 63%|██████▎   | 7043/11162 [28:11<26:30,  2.59it/s]

 63%|██████▎   | 7045/11162 [28:11<21:52,  3.14it/s]

 63%|██████▎   | 7047/11162 [28:12<19:26,  3.53it/s]

 63%|██████▎   | 7049/11162 [28:12<17:39,  3.88it/s]

 63%|██████▎   | 7050/11162 [28:12<14:33,  4.71it/s]

 63%|██████▎   | 7051/11162 [28:12<16:26,  4.17it/s]

 63%|██████▎   | 7052/11162 [28:13<14:23,  4.76it/s]

 63%|██████▎   | 7053/11162 [28:13<16:47,  4.08it/s]

 63%|██████▎   | 7055/11162 [28:13<16:13,  4.22it/s]

 63%|██████▎   | 7057/11162 [28:14<15:54,  4.30it/s]

 63%|██████▎   | 7059/11162 [28:14<15:20,  4.46it/s]

 63%|██████▎   | 7060/11162 [28:19<1:53:31,  1.66s/it]

 63%|██████▎   | 7069/11162 [28:20<1:20:12,  1.18s/it]

 63%|██████▎   | 7071/11162 [28:20<1:00:38,  1.12it/s]

 63%|██████▎   | 7073/11162 [28:20<46:26,  1.47it/s]  

 63%|██████▎   | 7075/11162 [28:21<37:49,  1.80it/s]

 63%|██████▎   | 7077/11162 [28:21<30:34,  2.23it/s]

 63%|██████▎   | 7

Error on ID 467554.




 64%|██████▍   | 7142/11162 [28:42<15:15,  4.39it/s]

 64%|██████▍   | 7143/11162 [28:42<13:05,  5.12it/s]

Error on ID 377753.
Error on ID 389051.




 64%|██████▍   | 7144/11162 [28:43<14:48,  4.52it/s]

 64%|██████▍   | 7145/11162 [28:43<15:21,  4.36it/s]

 64%|██████▍   | 7146/11162 [28:43<14:28,  4.62it/s]

 64%|██████▍   | 7147/11162 [28:43<16:56,  3.95it/s]

 64%|██████▍   | 7149/11162 [28:44<16:09,  4.14it/s]

 64%|██████▍   | 7150/11162 [28:44<13:20,  5.01it/s]

 64%|██████▍   | 7151/11162 [28:44<16:46,  3.99it/s]

 64%|██████▍   | 7152/11162 [28:45<14:29,  4.61it/s]

 64%|██████▍   | 7153/11162 [28:45<16:28,  4.05it/s]

 64%|██████▍   | 7154/11162 [28:45<14:40,  4.55it/s]

 64%|██████▍   | 7155/11162 [28:45<17:19,  3.85it/s]

 64%|██████▍   | 7157/11162 [28:46<16:57,  3.94it/s]

 64%|██████▍   | 7159/11162 [28:47<20:39,  3.23it/s]

 64%|██████▍   | 7161/11162 [28:47<16:21,  4.08it/s]

 64%|██████▍   | 7162/11162 [28:47<18:48,  3.54it/s]

 64%|██████▍   | 7164/11162 [28:48<17:39,  3.77it/s]

 64%|██████▍   | 7165/11162 [28:48<14:28,  4.60it/s]

 64%|██████▍   | 7166/11162 [28:48<17:24,  3.82it/s]

 64%|██████▍   | 7167/1116